In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("fashion-mnist_train.csv")

df.head()

labels = df.label
X = df.drop(["label"], axis = 1)
labels = labels.to_numpy()
y_label = np.zeros([60000, 10])
for i in range(len(labels)):
    y_label[i][labels[i]]= 1


from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

print(f"Shape of train set is {X.shape}")
print(f"Shape of train label is {labels.shape}")




Xtest = pd.read_csv("fashion-mnist_test.csv")
ytest = Xtest.label
Xtest = Xtest.drop('label', axis = 1)
Xtest = Xtest.to_numpy()

ytest_labels = np.zeros([len(ytest), 10])

for i in range(len(ytest)):
    ytest_labels[i][ytest[i]] = 1


class NeuralNet():
    def __init__(self, layers=[784, 8, 10], alpha = 0.01, num_iters = 100):
        self.params = self.init_weights()
        self.layers = layers
        self.alpha= alpha
        self.num_iters = num_iters
        self.loss = []
        self.sample_size = None
        self.X = None
        self.y = None
        
    def init_weights(self):
        input_layer=784
        hidden_1=8
        output_layer=10

        params = {
            'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
            'W2':np.random.randn(output_layer, hidden_1) * np.sqrt(1. / output_layer),
            'b1':np.random.randn(32,),
            'b2':np.random.randn(10,)
        }

        return params

        
    def relu(self,Z):
        return np.maximum(0,Z)
    
    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))
    
    def softmax(self, x, derivative=False):
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)
  
    
    
    
    def forward_propagation(self):
        
        Z1 = np.dot(self.params["W1"], self.X.T)
        A1 = self.sigmoid(Z1)
        Z2 = np.dot(self.params["W2"], A1)
        A2 = self.softmax(Z2)
     
        self.params['Z1'] = Z1
        self.params['Z2'] = Z2
        self.params['A1'] = A1
        self.params['A2'] = A2
        return A2
    
    def back_propagation(self, output):
        def dRelu(x):
            x[x<=0] = 0
            x[x>0] = 1
            return x
        weights_change = {}
        params = self.params

        error = 2 * (output - self.y) / output.shape[0] * self.softmax(params['Z2'], derivative=True)      
        weights_change['W2'] = np.outer(error, params['A1'])
        
        error = np.dot(params['W2'].T, error) * self.sigmoid(params['Z1'], derivative=True)
        weights_change['W1'] = np.outer(error, self.X)
        
        for key, value in weights_change.items():
           params[key] -= self.l_rate * value
        
        return weights_change

    
   
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.init_weights() 
         

        for i in range(self.num_iters):
            h = self.forward_propagation()
            self.back_propagation(h)
            
            
    def predict(self, X):
        Z1 = X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        pred = self.softmax(Z2)
        return np.round(pred) 
    
    def acc(self, y, h):
        acc = int(sum(y == h) / len(y) * 100)
        return acc


        
    

nn = NeuralNet(X, labels)
nn.fit(X, labels)

train_pred = nn.predict(X)
#test_pred = nn.predict(Xtest)

print("Train accuracy is {}".format(nn.acc(labels, train_pred)))
#print("Test accuracy is {}".format(nn.acc(ytest, test_pred)))


Shape of train set is (60000, 784)
Shape of train label is (60000,)


MemoryError: Unable to allocate 2.10 TiB for an array with shape (600000, 480000) and data type float64